# Import & Dataset 읽어오기

In [1]:
from datasets import concatenate_datasets, load_from_disk
from src.retriever.retrieval.sparse_retrieval import SparseRetrieval
from transformers import AutoTokenizer

org_dataset = load_from_disk('data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


# Retriever 선언 및 Retriever 진행
* 아마 초기엔 학습 진행하는데 시간 좀 걸릴예정
    * 약 15분정도 소요(승범 컴 기준)
    * 노션에 mrc/검증결과/Sparse 처리에 가장 좋은 bm25올려났으므로 그거 data 폴더 안에 넣으면 실행가능.

In [2]:
# 위에서 선언한거 가져오기 Retriever
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=False,)
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path="./data/",
    context_path="wikipedia_documents.json",
    mode = "bm25",
    max_feature=1000000,
    ngram_range=(1,2),
    #tokenized_docs = tokenized_docs,
)
retriever.get_sparse_embedding()

# Top k 조절.
df = retriever.retrieve(full_ds, topk=10)

# 여기서 df가 reader가 사용할 데이터 프레임입니다.
df.head(5)

Lengths of unique contexts : 56737
Building bm25 embedding...
Start Initializing...


Tokenizing...: 100%|██████████| 56737/56737 [02:22<00:00, 397.43it/s]


Generating n-grams and building vocabulary...


Generating n-grams: 100%|██████████| 56737/56737 [00:34<00:00, 1623.07it/s]


Current mode : bm25
End Initialization


Calculating BM25: 100%|██████████| 57/57 [23:35<00:00, 24.83s/it]


Finish BM25 Embedding
bm25 embedding shape: (56737, 1000000)
(4192, 1000000) (56737, 1000000)
<class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'>
result shape : (4192, 56737)
[query exhaustive search] done in 59.492 s


Sparse retrieval: 100%|██████████| 4192/4192 [00:00<00:00, 7737.26it/s]


,question,id,retrieval_context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 ...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"1943년 초, 마리아 발토르타가 9년 간 건강 상태가 안 좋아졌을 때, 그녀의 고...",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,양산 통도사의 금동천문도의 전면에는 천구(天球)의 북극을 중심으로 둥글게 북극으로 ...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


* question : 질문
* id : mrc 번호
* retrieval_context : retrieval 결과로 나온 topk를 하나의 text로 만든것
* original_context : 실제 retrieval 정답
* answers: dict 형태
    * answers_start : original_context에서 어디에서 시작하는지
    * text: 정답의 text값이 뭔지 

In [3]:
len(df['retrieval_context'][0])

13059

## Retriever 결과물 DatasetDict로 저장
reader는 데이터셋을 DatasetDict형태로 받으므로, df에서 변환이 필요하다

In [4]:
from datasets import Dataset, DatasetDict

# df를 Dataset으로
retriever_dataset = Dataset.from_pandas(df)

# DatasetDict로 변환 (train과 validation을 동일 데이터로 초기화)
dataset_dict = DatasetDict({
    'train': retriever_dataset,
    'validation': retriever_dataset
})

# train 스플릿: 'original_context'를 'context'로 사용
dataset_dict['train'] = dataset_dict['train'].remove_columns(['retrieval_context']).rename_column('original_context', 'context')

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict['validation'] = dataset_dict['validation'].remove_columns(['original_context']).rename_column('retrieval_context', 'context')

print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 4192
    })
    validation: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 4192
    })
})


In [5]:
# 별도 파일로 저장
dataset_dict.save_to_disk('outputs/bm25')

Saving the dataset (1/1 shards): 100%|██████████| 4192/4192 [00:00<00:00, 38603.03 examples/s]


# Reader

## do train

In [6]:
from transformers import HfArgumentParser, TrainingArguments
from datasets import DatasetDict
from src.reader.model.reader import Reader
from src.utils.arguments import DataTrainingArguments, ModelArguments

# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/models/train_dataset')

# 학습 관련 설정
training_args.do_train = True
training_args.do_eval = False
training_args.do_predict = False

# 학습 하이퍼파라미터 설정
model_args.model_name_or_path = 'klue/bert-base'  # 학습에 사용할 모델
training_args.learning_rate = 5e-5  # 학습률
training_args.num_train_epochs = 3  # epoch 수
training_args.per_device_train_batch_size = 16  # 학습 배치 사이즈

# 데이터셋 로드 (BM25 기반으로 생성된 데이터셋)
dataset = DatasetDict.load_from_disk('outputs/bm25')

# Reader 모델 학습 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-processing...


Map: 100%|██████████| 4192/4192 [00:04<00:00, 977.83 examples/s] 
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


preprocessed Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 8452
})


Step,Training Loss
500,1.371300
1000,0.457600
1500,0.171400


2024-10-23 23:21:02,029 - klue/bert-base - INFO - ***** Train results *****
2024-10-23 23:21:02,029 - klue/bert-base - INFO - epoch = 3.0
2024-10-23 23:21:02,030 - klue/bert-base - INFO - total_flos = 4969033471087536.0
2024-10-23 23:21:02,030 - klue/bert-base - INFO - train_loss = 0.6351578092905584
2024-10-23 23:21:02,030 - klue/bert-base - INFO - train_runtime = 552.9956
2024-10-23 23:21:02,031 - klue/bert-base - INFO - train_samples = 8452
2024-10-23 23:21:02,031 - klue/bert-base - INFO - train_samples_per_second = 45.852
2024-10-23 23:21:02,032 - klue/bert-base - INFO - train_steps_per_second = 2.87


***** train metrics *****
  epoch                    =        3.0
  total_flos               =  4627773GF
  train_loss               =     0.6352
  train_runtime            = 0:09:12.99
  train_samples            =       8452
  train_samples_per_second =     45.852
  train_steps_per_second   =       2.87


## do eval

In [7]:
# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/train_dataset')

# 평가 관련 설정
training_args.do_train = False
training_args.do_eval = True
training_args.do_predict = False

# 학습된 모델 경로 설정
model_args.model_name_or_path = 'outputs/models/train_dataset'

# 평가 데이터셋 로드 (BM25 기반으로 생성된 데이터셋)
dataset = DatasetDict.load_from_disk('outputs/bm25')

# Reader 모델 평가 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()


Pre-processing...


Map: 100%|██████████| 4192/4192 [01:21<00:00, 51.65 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


preprocessed Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 109120
})


/opt/conda/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:483: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


postprocess_qa Dataset({
    features: ['question', 'id', 'context', 'answers'],
    num_rows: 4192
})


100%|██████████| 4192/4192 [05:35<00:00, 12.49it/s]
2024-10-23 23:45:18,562 - postprocess log - INFO - Saving predictions to outputs/train_dataset/predictions.json.
2024-10-23 23:45:18,570 - postprocess log - INFO - Saving nbest_preds to outputs/train_dataset/nbest_predictions.json.
2024-10-23 23:45:20,409 - outputs/models/train_dataset - INFO - Evaluation results: {'exact_match': 78.74522900763358, 'f1': 80.33898883135521}


## do predict

In [8]:
# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/test_dataset')

# 추론 관련 설정
training_args.do_train = False
training_args.do_eval = False
training_args.do_predict = True

# 학습된 모델 경로 및 테스트 데이터셋 경로 설정
model_args.model_name_or_path = 'outputs/models/train_dataset'
data_args.dataset_name = 'data/test_dataset'  # 추론에 사용할 테스트 데이터셋 경로

# 테스트 데이터셋 로드
dataset = DatasetDict.load_from_disk('data/test_dataset')

# Reader 모델 추론 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()


Pre-processing...


Map: 100%|██████████| 600/600 [00:00<00:00, 13286.85 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


preprocessed Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 600
})


/opt/conda/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:483: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


postprocess_qa Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 600
})


KeyError: "Column id not in the dataset. Current columns in the dataset: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id']"